# Fine-tune BLIP using Hugging Face `transformers` and `datasets` 🤗

This tutorial is largely based from the [GiT tutorial](https://colab.research.google.com/drive/1HLxgrG7xZJ9FvXckNG61J72FkyrbqKAA?usp=sharing) on how to fine-tune GiT on a custom image captioning dataset. Here we will use a dummy dataset of [football players](https://huggingface.co/datasets/ybelkada/football-dataset) ⚽ that is uploaded on the Hub. The images have been manually selected together with the captions. 
Check the 🤗 [documentation](https://huggingface.co/docs/datasets/image_dataset) on how to create and upload your own image-text dataset.

In [1]:
# install requirements
import sys

!pip3 install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
# !git clone https://github.com/salesforce/BLIP
!pip install pycocotools pycocoevalcap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 22.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.5 MB/s eta 0:00:00:00:01
  Created wheel for fairscale: filename=fairscale-0.4.4-py3-none-any.whl size=292837 sha256=535e5c322a1e9ac9852fabdc4a7a09eb1b76d6633e6db0bb403dd50303c0185b
  Stored in directory: /root/.cache/pip/wheels/51/77/ae/9576097dae2412e61bc5435e3df7ab24df98e9e43ff6d2c07d
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e64ec4

In [2]:
# ! wget https://github.com/salesforce/BLIP/archive/refs/heads/main.zip
# ! unzip main.zip
# ! cp -r BLIP-main/* /kaggle/working/

In [3]:
# %cd /kaggle/working
# ! git clone https://github.com/MahmoudQaid/Blip
# %cd Blip

In [4]:
from tqdm.notebook import tqdm

# train_caption.py

In [16]:
def u():
    %cd /kaggle/working
    ! rm -r Blip
    ! git clone https://github.com/MahmoudQaid/Blip
    %cd Blip
u()

/kaggle/working
Cloning into 'Blip'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 162 (delta 91), reused 69 (delta 25), pack-reused 0
Receiving objects: 100% (162/162), 6.97 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/kaggle/working/Blip


In [20]:
from collections import defaultdict
import json,string,re

masked_word=['is','at','for','am','are','and','or','but','of']
pattern=r'\b('+'|'.join(masked_word)+r')\b'

def load_doc_karpathy(json_file_path,pattern=None,lower=False):
    # this function return dictionary 
    with open(json_file_path,'r') as file:
        data=json.loads(file.read())
    dict_data=defaultdict(list)
    for example in data['images']:
        temp=[]
        
        for sentence in example['sentences']:
            cap=sentence['raw']
            if lower:
                cap=cap.lower()
                
            cap=cap.translate(str.maketrans('','',string.punctuation))
            
            if pattern is not None:
                cap=re.sub(pattern,'',cap)
            cap=' '.join(cap.split())
            if example['split']=='train':
                dict_data[example['split']].append({'caption':cap,
                                                    'image': example['filename'],
                                                    'image_id': example['imgid']})
            else:
                temp.append(cap)
                
        if example['split']!='train':
            dict_data[example['split']].append({'caption':temp,
                                                'image': example['filename']
                                             })
            
# {'train':[{'image':'name.jpg','image_id':img_id,'caption':cap}],'test':[{'image':'name.jpg','captions':[cap1,...,cap5]}],'val':[{'image':'name.jpg','captions':[cap1,...,cap5]}]}  
    return dict_data


KARPATHY_DATA=load_doc_karpathy(json_file_path='/kaggle/input/karpathy-splits/dataset_flickr8k.json',pattern=pattern,lower=True)   


In [7]:
def prepare_ref_caps_for_evaluate(list_dict,output_json_path):
    references={'annotations':[],
                'images':[]}
    j=0
    for i,d in enumerate(list_dict):
        for cap in d['caption']:
            references['annotations'].append({'image_id':d['image'],'caption':cap,'id':j})
            j+=1
        references['images'].append({'id':d['image']})
    json.dump(references,open(output_json_path,'w'))
    
    return references
    


In [8]:
len(KARPATHY_DATA['train'])

30000

In [9]:
# New=defaultdict(list)
# for i in KARPATHY_DATA:
#     for j,d in enumerate(KARPATHY_DATA[i]):
#         if j ==100:
#             break
#         New[i].append(d)
# # d
# KARPATHY_DATA=dict(New)


In [10]:
# ! mkdir annotation
# json.dump(KARPATHY_DATA['val'],open('annotation/coco_karpathy_val_gt.json','w'))
# json.dump(KARPATHY_DATA['test'],open('annotation/coco_karpathy_test_gt.json','w'))
# json.dump(KARPATHY_DATA['train'],open('annotation/coco_karpathy_train_gt.json','w'))

In [11]:
# ! pip install vit_pytorch
# import torch
# from vit_pytorch.vit import ViT

# v = ViT(
#     image_size = 256,
#     patch_size = 32,
#     num_classes = 1000,
#     dim = 1024,
#     depth = 6,
#     heads = 16,
#     mlp_dim = 2048,
#     dropout = 0.1,
#     emb_dropout = 0.1
# )

# # forward pass now returns predictions and the representation of the final layer
# predictions = v(torch.randn(1, 3, 256, 256))
# predictions.shape

In [21]:
model=None

In [ ]:
'''
 * Copyright (c) 2022, salesforce.com, inc.
 * All rights reserved.
 * SPDX-License-Identifier: BSD-3-Clause
 * For full license text, see LICENSE.txt file in the repo root or https://opensource.org/licenses/BSD-3-Clause
 * By Junnan Li
'''
import argparse
import os
import ruamel_yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.distributed as dist
from torch.utils.data import DataLoader

from models.blip import blip_decoder
import utils
from utils import cosine_lr_schedule
from data import create_dataset, create_sampler, create_loader
from data.utils import save_result, coco_caption_eval


# dist.init_process_group(backend='nccl',world_size=1, init_method='env://',rank=0)


def train(model, data_loader, optimizer, epoch, device):
    # train
    model.train()  
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('loss', utils.SmoothedValue(window_size=1, fmt='{value:.4f}'))
    header = 'Train Caption Epoch: [{}]'.format(epoch)
    print_freq = 50

    for i, (image, caption, _) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        image = image.to(device)       
        
        loss = model(image, caption)      
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger.global_avg())     
    return {k: "{:.3f}".format(meter.global_avg) for k, meter in metric_logger.meters.items()}  


@torch.no_grad()
def evaluate(model, data_loader, device, config):
    # evaluate
    model.eval() 
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Caption generation:'
    print_freq = 10

    result = []
    for image, image_id in metric_logger.log_every(data_loader, print_freq, header): 
        
        image = image.to(device)       
        
        captions = model.generate(image, sample=False, num_beams=config['num_beams'], max_length=config['max_length'], 
                                  min_length=config['min_length'])
        
        for caption, img_id in zip(captions, image_id):
#             result.append({"image_id": img_id.item(), "caption": caption})
            result.append({"image_id": img_id, "caption": caption})
  
    return result


def main(args, config,model=None):
    utils.init_distributed_mode(args)    
    
    device = torch.device(args.device)

    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.benchmark = True

    #### Dataset #### 
    print("Creating captioning dataset")
    train_dataset, val_dataset, test_dataset = create_dataset('caption_coco', config,annot_dict=KARPATHY_DATA)  
#     train_dataset, val_dataset, test_dataset = create_dataset('caption_coco', config)  

    if args.distributed:
        num_tasks = utils.get_world_size()
        global_rank = utils.get_rank()            
        samplers = create_sampler([train_dataset,val_dataset,test_dataset], [True,False,False], num_tasks, global_rank)         
    else:
        samplers = [None, None, None]
    
    train_loader, val_loader, test_loader = create_loader([train_dataset, val_dataset, test_dataset],samplers,
                                                          batch_size=[config['batch_size']]*3,num_workers=[4,4,4],
                                                          is_trains=[True, False, False], collate_fns=[None,None,None])         

    start_epoch=0
    max_epoch=config['max_epoch']
    #### Model #### 
    print("Creating model")
    if config['load_pretrained'] and config['pretrained']:
        checkpoint=torch.load(config['pretrained'])
        model = blip_decoder(pretrained=checkpoint['model'], image_size=config['image_size'], vit=config['vit'], 
                               vit_grad_ckpt=config['vit_grad_ckpt'], vit_ckpt_layer=config['vit_ckpt_layer'], 
                               prompt=config['prompt'])
        max_epoch=checkpoint['epoch']

        start_epoch=checkpoint['epoch']+1
        max_epoch=start_epoch+config['max_epoch']
#         model.load_state_dict(checkpoint['model'])
    elif model is None:
        model = blip_decoder(pretrained='', image_size=config['image_size'], vit=config['vit'], 
                               vit_grad_ckpt=config['vit_grad_ckpt'], vit_ckpt_layer=config['vit_ckpt_layer'], 
                               prompt=config['prompt'])
        
    for param in model.visual_encoder.parameters():
        param.requires_grad=False

    model = model.to(device)   
    model_without_ddp = model
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        model_without_ddp = model.module 
        
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=config['init_lr'], weight_decay=config['weight_decay'])
    if config['load_pretrained'] and config['pretrained']:
        optimizer.load_state_dict(checkpoint['optimizer'])  
        
    best = 0
    best_epoch = 0

    print("Start training")
    start_time = time.time()    
    for epoch in range(start_epoch, max_epoch):
        if not args.evaluate:        
            if args.distributed:
                train_loader.sampler.set_epoch(epoch)
                
            cosine_lr_schedule(optimizer, epoch, config['max_epoch'], config['init_lr'], config['min_lr'])
            train_stats = train(model, train_loader, optimizer, epoch, device) 
        val_result = evaluate(model_without_ddp, val_loader, device, config)
        val_result_file = save_result(val_result, args.result_dir, 'val_epoch%d'%epoch, remove_duplicate='image_id')        
    
        test_result = evaluate(model_without_ddp, test_loader, device, config)  
        test_result_file = save_result(test_result, args.result_dir, 'test_epoch%d'%epoch, remove_duplicate='image_id')  
        
        if utils.is_main_process():
            prepare_ref_caps_for_evaluate(KARPATHY_DATA['val'],config['ann_root']+'/coco_karpathy_val_gt.json')
            prepare_ref_caps_for_evaluate(KARPATHY_DATA['test'],config['ann_root']+'/coco_karpathy_test_gt.json')
            coco_val = coco_caption_eval(config['ann_root'],val_result_file,'val')
            coco_test = coco_caption_eval(config['ann_root'],test_result_file,'test')
            if args.evaluate:            
                log_stats = {**{f'val_{k}': v for k, v in coco_val.eval.items()},
                             **{f'test_{k}': v for k, v in coco_test.eval.items()},                       
                            }
                with open(os.path.join(args.output_dir, "evaluate.txt"),"a") as f:
                    f.write(json.dumps(log_stats) + "\n")                   
            else:             
                save_obj = {
                    'model': model_without_ddp.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'config': config,
                    'epoch': epoch,
                }

                if coco_val.eval['CIDEr'] + coco_val.eval['Bleu_4'] > best:
                    best = coco_val.eval['CIDEr'] + coco_val.eval['Bleu_4']
                    best_epoch = epoch                
                    torch.save(save_obj, os.path.join(args.output_dir, 'checkpoint_best.pth')) 
                    
                log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                             **{f'val_{k}': v for k, v in coco_val.eval.items()},
                             **{f'test_{k}': v for k, v in coco_test.eval.items()},                       
                             'epoch': epoch,
                             'best_epoch': best_epoch,
                            }
                with open(os.path.join(args.output_dir, "log.txt"),"a") as f:
                    f.write(json.dumps(log_stats) + "\n")     
                    
        if args.evaluate: 
            break
#         dist.barrier()     

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str)) 
    return model
class Args(dict):
    __setattr__ = dict.__setitem__
    __getattr__ = dict.__getitem__

args = {
    'config':'./configs/caption_coco.yaml',
    'output_dir':'output/Caption_coco' ,
    'evaluate':False ,
    'device':'cuda',
    'seed':42,
    'world_size':1,  
    'dist_url':'env://',
    'distributed':False
}
args = Args(args) # dict2object

if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--config', default='./configs/caption_coco.yaml')
#     parser.add_argument('--output_dir', default='output/Caption_coco')        
#     parser.add_argument('--evaluate', action='store_true')    
#     parser.add_argument('--device', default='cuda')
#     parser.add_argument('--seed', default=42, type=int)
#     parser.add_argument('--world_size', default=1, type=int, help='number of distributed processes')    
#     parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
#     parser.add_argument('--distributed', default=True, type=bool)
#     args = parser.parse_args()

#     config = yaml.load(open(args.config, 'r'), Loader=yaml.Loader)
    config={
     'image_root': '/kaggle/input/flickr8k/Images',
#      'image_root':'/kaggle/input/coco2014',
     'ann_root': 'annotations',
     'coco_gt_root': 'annotation/coco_gt',
#      'pretrained':''
#      'pretrained': 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth', # vit: base
#      'pretrained': 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth',  # vit: large
     'pretrained':'/kaggle/input/blip-temp/Blip/output/Caption_coco/checkpoint_best.pth',
     'vit': 'base',
#      'vit': 'large',
     'vit_grad_ckpt': False,
     'vit_ckpt_layer': 0,
     'batch_size': 16,
     'init_lr': 1e-05,
     'image_size': 384,
     'max_length': 20,
     'min_length': 5,
     'num_beams': 3,
     'prompt': 'a picture of ',
     'weight_decay': 0.05,
     'min_lr': 0,
     'max_epoch': 2,
     'load_pretrained':True
    }
    args.result_dir = os.path.join(args.output_dir, 'result')

    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    Path(args.result_dir).mkdir(parents=True, exist_ok=True)
    Path(config['ann_root']).mkdir(parents=True, exist_ok=True)
    
    yaml.dump(config, open(os.path.join(args.output_dir, 'config.yaml'), 'w'))    

    model=main(args,config,model)

Not using distributed mode
Creating captioning dataset
Creating model
Start training
Train Caption Epoch: [1]  [   0/1875]  eta: 0:55:57  lr: 0.000005  loss: 3.1561  time: 1.7905  data: 1.1600  max mem: 9722
Train Caption Epoch: [1]  [  50/1875]  eta: 0:16:16  lr: 0.000005  loss: 3.0375  time: 0.5154  data: 0.0007  max mem: 10426
Train Caption Epoch: [1]  [ 100/1875]  eta: 0:15:19  lr: 0.000005  loss: 3.2467  time: 0.5029  data: 0.0002  max mem: 10426
Train Caption Epoch: [1]  [ 150/1875]  eta: 0:14:45  lr: 0.000005  loss: 3.0145  time: 0.5015  data: 0.0002  max mem: 10591
Train Caption Epoch: [1]  [ 200/1875]  eta: 0:14:17  lr: 0.000005  loss: 2.8578  time: 0.5004  data: 0.0001  max mem: 10591
Train Caption Epoch: [1]  [ 250/1875]  eta: 0:13:48  lr: 0.000005  loss: 2.9157  time: 0.5039  data: 0.0002  max mem: 10591
Train Caption Epoch: [1]  [ 300/1875]  eta: 0:13:20  lr: 0.000005  loss: 3.0737  time: 0.5050  data: 0.0004  max mem: 10591
Train Caption Epoch: [1]  [ 350/1875]  eta: 0:12

In [ ]:
# from torchsummary import summary
# summary(model,)


In [ ]:
assert(False)

In [ ]:
from glob import glob
glob('output/Caption_coco/result/*')

In [ ]:
# chpt['model'].keys()

In [ ]:
# visual_dict.keys()

In [ ]:
# import torch 
from torchvision.datasets.utils import download_url
download_url('https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth','/kaggle/working')
chpt=torch.load('/kaggle/working/model_large_caption.pth')
visual_dict={}
for i in chpt['model']:
    if i.split('.')[0]=='visual_encoder':
        visual_dict[i[len('visual_encoder.'):]]=chpt['model'][i]
model.visual_encoder.load_state_dict(visual_dict)

In [ ]:
# download_url('https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth','/kaggle/working')
# chpt2=torch.load('/kaggle/working/model_base_caption_capfilt_large.pth')

In [ ]:
# # len(chpt2['model'].keys()),len(chpt['model'].keys())
# for i,key in enumerate(list(chpt['model'].keys())):
#     if key not in list(chpt2['model'].keys()):
#         print(i," : ",key)

In [ ]:
assert(False)

In [ ]:
KARPATHY_DATA['val'][0],KARPATHY_DATA['train'][0]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def eval_train_split():
    train_kar=[]
    train_img=[]
    for i in KARPATHY_DATA['train']:
        if i['image'] in train_img:
            continue
        train_img.append(i['image'])
        train_kar.append({'image':i['image'],'caption':[i['caption']]})

    train_dataset, val_dataset, test_dataset = create_dataset('caption_coco', config,annot_dict={'val':train_kar,'train':KARPATHY_DATA['train'],'test':KARPATHY_DATA['val']})  


    samplers = [None, None, None]

    train_loader, val_loader, test_loader = create_loader([train_dataset, val_dataset, test_dataset],samplers,
                                                          batch_size=[config['batch_size']]*3,num_workers=[4,4,4],
                                                          is_trains=[True, False, False], collate_fns=[None,None,None])         
    train_result=evaluate(model, val_loader, device, config)
    json.dump(train_result,open('/kaggle/working/train_result.json','w'))
    print(train_result[0])
    
    
eval_train_split()


# Try model generation

In [ ]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 384
def load_demo_image(image_size,device): 
    raw_image = Image.open(img_url).convert('RGB')   

    w,h = raw_image.size
    display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image



In [ ]:
# beam search
img_url="/kaggle/input/flickr8k/Images/"+KARPATHY_DATA['test'][8]['image']
image=load_demo_image(image_size,device)
caption = model.generate(image, sample=False, num_beams=3, max_length=20, min_length=5) 
# nucleus sampling
# caption = model.generate(image, sample=True, top_p=0.9, max_length=20, min_length=5) 
print('caption: '+caption[0])



In [ ]:
assert(False)

# HappyTT 

In [ ]:
from collections import defaultdict
import json,string,re

masked_word=['is','at','for','am','are','and','or','but','of']
pattern=r'\b('+'|'.join(masked_word)+r')\b'

def prepare_tt_kar_text(json_file_path,pattern=None,lower=False):
    # this function return dictionary 
    with open(json_file_path,'r') as file:
        data=json.loads(file.read())
    dict_data=defaultdict(lambda: defaultdict(list))
    for example in data['images']:
        temp=[]
        for sentence in example['sentences']:
            cap=sentence['raw']
            if lower:
                cap=cap.lower()
                
            cap=cap.translate(str.maketrans('','',string.punctuation))
            
            if pattern is not None:
                cap=re.sub(pattern,'',cap)
            cap=' '.join(cap.split())
            
            dict_data[example['split']][example['filename']].append(cap)

            
# {'train':[{'image':'name.jpg','image_id':img_id,'caption':cap}],'test':[{'image':'name.jpg','captions':[cap1,...,cap5]}],'val':[{'image':'name.jpg','captions':[cap1,...,cap5]}]}  
    return dict_data
kar_with_pattern=prepare_tt_kar_text('/kaggle/input/karpathy-splits/dataset_flickr8k.json',pattern=pattern,lower=True)
raw_kar=prepare_tt_kar_text('/kaggle/input/karpathy-splits/dataset_flickr8k.json',pattern=None,lower=False)

In [ ]:
# train_result=json.load(open('/kaggle/input/blip-mask-results-1e/train_result.json','r'))
# val_result=json.load(open('/kaggle/input/blip-mask-results-1e/val_result.json','r'))
# test_result=json.load(open('/kaggle/input/blip-mask-results-1e/test_result.json','r'))

# len(train_result),len(val_result),len(test_result),train_result[0],val_result[0],test_result[0]

In [ ]:
for i in raw_kar['train']:
    print(raw_kar['train'][i])
    print()
    print(kar_with_pattern['train'][i])
    break

In [ ]:
import csv
captioning_results={
    'train':json.load(open('/kaggle/working/train_result.json','r')),
    'val':json.load(open('output/Caption_coco/result/val_epoch0.json','r')),
    'test':json.load(open('output/Caption_coco/result/test_epoch0.json','r'))
}
def generate_csv(csv_path, splits=['train'],input_mode={'pred':True,'masked':True}):
    # input_dict: {'img_name':['one caption']}
    # refs_dict: {'img_name':['caption1','cap2','cap3']}
    
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for sp in splits:
            if input_mode['masked']:
                for img in kar_with_pattern[sp]:
                    for i in range(len(kar_with_pattern[sp][img])):
                        writter.writerow(['grammar: '+kar_with_pattern[sp][img][i], raw_kar[sp][img][i]])
            if input_mode['pred']:
                for res in captioning_results[sp]:
                    for cap in raw_kar[sp][res['image_id']]:
                        writter.writerow(['grammar: '+res['caption'], cap])


In [ ]:

generate_csv("train.csv", splits=['train'],input_mode={'pred':True,'masked':True})
generate_csv("eval.csv", splits=['val'],input_mode={'pred':True,'masked':False})
generate_csv("test.csv", splits=['test'],input_mode={'pred':True,'masked':False})

In [ ]:
# import pandas as pd
# df=pd.read_csv('train.csv')
# len(df)

In [ ]:
# df.head(20)

In [ ]:
!pip install happytransformer

from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
# happy_tt = HappyTextToText("T5", "t5-base")


args = TTSettings(num_beams=5, min_length=1)

# Add the prefix "grammar: " before each input 
result = happy_tt.generate_text("grammar: This sentences has has bads grammar.", args=args)
print(result.text) # This sentence has bad grammar.

In [ ]:
from happytransformer import TTTrainArgs
args2 = TTTrainArgs(batch_size=50)
happy_tt.train("train.csv", args=args2)

# saving model

In [ ]:
happy_tt.save("/kaggle/working/happy_tt/")

In [ ]:
before_training=happy_tt.eval('eval.csv')
print(before_training.loss)

In [ ]:
preds2=[]
annot={'annotations':[],'images':[]}
for i,res in enumerate(tqdm(captioning_results['test'])):
    out_cap=happy_tt.generate_text("grammar: "+res['caption'], args=args).text
    preds2.append({'image_id':res['image_id'],'caption':out_cap})
    for cap in raw_kar['test'][res['image_id']]:
        annot['images'].append({'id':res['image_id']})
        annot['annotations'].append({'image_id':res['image_id'],'caption':cap,'id':i})
        

In [ ]:
json.dump(annot,open('/kaggle/working/coco_karpathy_test_gt.json','w'))
json.dump(preds2,open('/kaggle/working/coco_karpathy_test_result.json','w'))
coco_val = coco_caption_eval('/kaggle/working','/kaggle/working/coco_karpathy_test_result.json','test')

In [ ]:
assert(False)

In [ ]:
json.dump(annot,open('/kaggle/working/coco_karpathy_test_gt.json','w'))
json.dump(preds2,open('/kaggle/working/coco_karpathy_test_result.json','w'))
coco_val = coco_caption_eval('/kaggle/working','/kaggle/working/coco_karpathy_test_result.json','test')
